In [1]:
# importazione delle librerie
import pandas as pd
import pandas_ta as ta
import numpy as np
import funzioni as fn

# importiamo il path
from funzioni import PATH, PATH_ESTR
from itertools import combinations, permutations

CREIAMO LA NOSTRA CLASSE PER IL GIOCO DEL LOTTO

In [2]:
# funzione per dire quando e' uscito un numero
def uscite(df, numero):
    # prendiamo l'indice della riga che contiene un valore in una determinata colonna
    # la variabile numero e' una lista di numeri
    numeri = np.array(numero)
    uscite = np.array([])
    if len(numeri) == 1:
        for i in df.columns[2:]:
            # aggiungi l'indice della riga che contiene il valore 58 nella colonna i
            uscite = np.append(uscite, df[df[i] == str(numeri[0])].index.values.astype(int))
    elif len(numeri) > 1:

        for i in range(len(df)):
            # prendiamo la riga i-esima con le colonne dalla 2 alla 8
            riga = df.iloc[i, 2:]
            
            # preniamo tutte le possibili combianzioni di numeri e le mettiamo in una lista
            # e poi verifichiamo se sono uscite
            for j in permutations(riga, len(numeri)):
                
                # convertiamo l´array numeri in una turpla di stringhe
                numeri = tuple(map(str, numeri))
                if j == numeri:
                    uscite = np.append(uscite, i)
                     
    
    # aggiungiamo ad ogni valore della lista il valore 1
    #uscite = np.sort(uscite + 1)
    uscite = np.sort(uscite)
    return uscite


In [3]:
def ritardi(df, numero):

    u = uscite(df, numero)
    # calcuiamo la differenza tra ogni valore della lista e il successivo
    ritardi = np.diff(u)
    # inseriamo al primo posto il valore 0
    ritardi = np.insert(ritardi, 0, 0)

    return ritardi

In [4]:
class Lotto:

    def __init__(self, path=PATH_ESTR):
        """Di default il path e' la cartella estrazioni"""
        self.ruota = None
        self.df = None
        self.numero = None
        self.path = path

    def carica_estrazioni(self, ruota):
        """Carica le estrazioni di una ruota in un dataframe"""
        r = self.path+ruota+'.csv'
        estrazioni = fn.leggi_da_csv(r)
        df = pd.DataFrame(estrazioni[1:], columns=estrazioni[0])
        # eliminiamo le righe duplicate
        df.drop_duplicates(inplace=True)
        self.ruota = ruota
        return df

    def carica_numero(self, numero, df, n_estr=10):
        """
        Elabora le informazioni statistiche sul numero e sui numeri inseiriti
        Il numero va inserito come [] di numeri interi
        """
        self.df = df
        self.n_estr = n_estr
        self.numero = numero
        # il totale delle estrazioni e' dato dalla lunghezza del dataframe
        self.totale_estrazioni = len(self.df)
        # calcoliamo le posizioni di indice delle uscite del numero
        self.elenco_uscite = uscite(self.df, self.numero)
        # il numero totali delle volte che e' uscito e' dato dalla lunghezza della lista
        self.n_uscite = len(self.elenco_uscite)
        if self.n_uscite > 0:
            # il vettore delle frequenze e' dato dal vettore del numero di uscite diviso il vettore dell'indice del vettore stesso + 1
            self.frequenze = self.elenco_uscite / (np.arange(len(self.elenco_uscite))+1)
            # la frequenza media (ultima frequenza o frequenza attuale) e' data dal totale delle estrazioni diviso il numero di uscite
            self.ultima_frequenza = round(float(self.totale_estrazioni/self.n_uscite), 2)
            # l'elenco dei ritardi e' dato dal vettore dei ritardi
            self.ritardi = ritardi(self.df, self.numero)
            # il ritardo attuale e' dato dalla differenza tra il totale delle estrazioni e l'indice l'ultima uscita
            self.ritardo_attuale = self.totale_estrazioni - self.elenco_uscite[-1]
            # calcoliamo il ritardo massimo storico
            self.ritardo_massimo = self.ritardi.max()
            # salviamo l'ultimo ritardo
            self.ultimo_ritardo = self.ritardi[-1]
            # calcoliamo la scompensazione
            self.scompensazioni = self.ritardi - self.frequenze
            # calcoliamo la scompensazione attuale e l'ultima scompensazione
            self.ultima_scompensazione = self.scompensazioni.sum()
            self.scompensazione_attuale = self.ultima_scompensazione + (self.ritardo_attuale - self.ultima_frequenza)     
            # calcoliamo le spie
            self.spia_num, self.spia_rip = self.spia()
            # salviamo le statistiche in un dizionario
        else:
            self.frequenze = np.NaN
            self.ultima_frequenza = np.NaN
            self.ritardi = np.NaN
            self.ritardo_attuale = np.NaN
            self.ritardo_massimo = np.NaN
            self.ultimo_ritardo = np.NaN
            self.scompensazioni = np.NaN
            self.ultima_scompensazione = np.NaN
            self.scompensazione_attuale = np.NaN
            self.spia_num, self.spia_rip, self.spia_estr = np.NaN, np.NaN, np.NaN
            print(f'{self.numero}, non è mai uscito')

        self.statistiche = {
            'ruota': self.ruota,
            'numero': self.numero,
            'n_uscite': self.n_uscite,
            'totale_estrazioni': self.totale_estrazioni,
            'ultima_frequenza': self.ultima_frequenza,
            'ritardo_attuale': self.ritardo_attuale,
            'ritardo_massimo': self.ritardo_massimo,
            'ultimo_ritardo': self.ultimo_ritardo,
            'ultima_scompensazione': self.ultima_scompensazione,
            'scompensazione_attuale': self.scompensazione_attuale,
            'spia_num': self.spia_num,
            'spia_rip': self.spia_rip,
            'spia_estr': self.n_estr
        }
        
    def stampa_statistihe(self):
        
        for k, v in self.statistiche.items():
            print(f'{k}: {v}')

    def spia(self):
        t = np.array([])
        for i in self.elenco_uscite:
            i = int(i) - 2
            t = np.append(t, self.df.iloc[i-self.n_estr:i, 2:].values)
        pf = pd.DataFrame(columns=['n_spia', 'rip'])
        pf['n_spia'], pf['rip'] = np.unique(t.flatten(), return_counts=True)
        pf.sort_values(by=['rip'], inplace=True)
        u, c = pf.iloc[-1]
        return u, c

    def calcolo_trend(self):

        l10 = 10
        l30 = 30 

        try:
            # utilizzo le funzioni di pandas_ta per calcolare il trend della scompensazione
            df = pd.DataFrame()
            df['scompensi'] = self.scompensazioni
            # aggiungo la colonna close perche' la funzione ta.sma la richiede
            df['close'] = df['scompensi'].cumsum()
            sma10 = df.ta.sma(length=l10)
            sma30 = df.ta.sma(length=l30)
            df = pd.concat([df, sma10, sma30], axis=1)
            df['scomp_sigLine'] = np.where(df['SMA_10'] > df['SMA_30'], 1, 0 )
            df['scomp_position'] = df['scomp_sigLine'].diff()
            df["Status"] = np.where(df['scomp_sigLine'] == 0, 'F', 'R')
            self.trend = df['Status'].iloc[-1]
            self.statistiche['trend'] = self.trend
        except:
            print('Impossibile calcolare il trend')
            self.statistiche['trend'] = np.NaN

    def previsioni(self, ruota):

        temp = self.carica_estrazioni(ruota)
        df = pd.DataFrame()

        # calcoliamo le previsioni
        for i in range(1, 91):

            self.carica_numero(i, temp)
            self.calcolo_trend()
            df = df.append(self.statistiche, ignore_index=True)
        
        df = df[(df['ritardo_attuale'] < 15) & (df['scompensazione_attuale'] < 0) & (df['trend'] == 'R')].sort_values(by='ultima_frequenza', ascending=False)
        self.previsione = df['numero'].values
        
        return f"i numeri da giocare su {ruota} sono: {self.previsione}"
        
            


In [5]:
def calcola_uscita_da_spia(spia):
    temp = Lotto()
    df_temp = temp.carica_estrazioni('BA')
    temp.carica_numero([spia], df_temp, 20)
    x = int(temp.elenco_uscite[-1]) - 1

    return len(df_temp) - x



In [6]:
# creiamo una lista in numpy con tutti gli ambi in 90 numeri
numeri = np.arange(1,91)
ambi = np.array(list(combinations(numeri,2)))
print(len(ambi))

4005


In [9]:
# creiamo in dataframe contenenti tutti gli ambi e il loro numero spia a 5, 10 e 20 estrazioni
dataframe = pd.DataFrame(columns=['Ambi', 'uscite', 'Spia10', 'rip10', 'tra10','Spia20', 'rip20', 'tra20'])
n = Lotto()
estrazioni = n.carica_estrazioni('BA')

for i in ambi[:100]:
    i = list(map(int, i))
    n.carica_numero(i, estrazioni, 10)
    n_uscite = n.n_uscite
    spia10 = n.spia_num
    rip10 = n.spia_rip
    tra10 = 10 - calcola_uscita_da_spia(spia10)
    n.carica_numero(i, estrazioni, 20)
    spia20 = n.spia_num
    rip20 = n.spia_rip
    tra20 = 20 - calcola_uscita_da_spia(spia20)
    dataframe = dataframe.append({
        'Ambi': i, 
        'uscite': n_uscite, 
        'Spia10': spia10, 'rip10': rip10, 'tra10': tra10,
        'Spia20': spia20, 'rip20': rip20, 'tra20': tra20
        }, ignore_index=True)

dataframe.head(100)

KeyboardInterrupt: 

In [ ]:
ambo = Lotto()
df = ambo.carica_estrazioni('BA')
ambo.carica_numero([22, 11], df, 20)
ambo.calcolo_trend()
ambo.stampa_statistihe()


In [ ]:
n = Lotto()
df = n.carica_estrazioni('BA')
numeri = pd.DataFrame(columns=['numero', 'spia', 'rip', 'uscite'])
num = np.array([])
sp = np.array([])
rip = np.array([])
u = np.array([])
for i in range(1, 91):
    n.carica_numero([i], df)
    num = np.append(num, int(i))
    sp = np.append(sp, n.spia_num)
    rip = np.append(rip, n.spia_rip)
    u = np.append(u, n.n_uscite)
numeri['numero'] = num
numeri['spia'] = sp
numeri['rip'] = rip
numeri['uscite'] = u



In [ ]:
n = Lotto()
df = n.carica_estrazioni('BA')
numeri = pd.DataFrame(columns=['numero', 'spia', 'rip', 'uscite'])

sp = np.array([])
rip = np.array([])
u = np.array([])
a = combinations(range(1, 91), 2)
a = list(a)
count= 1
for i in a:
    i = list(map(int, i))
    n.carica_numero(i, df)
    sp = np.append(sp, n.spia_num)
    rip = np.append(rip, n.spia_rip)
    u = np.append(u, n.n_uscite)
    fn.progress(count, len(a))
    count += 1
    
numeri['numero'] = a
numeri['spia'] = sp
numeri['rip'] = rip
numeri['uscite'] = u

In [ ]:
numeri['%'] = round(numeri['rip'] / numeri['uscite'], 2) * 100
lista_num = [59,14,40,79,7]
for i in lista_num:
    print(numeri[numeri['spia'] == str(i)])

In [ ]:
b = Lotto()
df = b.carica_estrazioni('BA')
b.carica_numero([1, 4], df)
b.calcolo_trend()
b.stampa_statistihe()



In [ ]:
# leggiamo il file csv delle estrazioni di Bari
a = Lotto()

ruote = ['BA', 'CA', 'FI', 'GE', 'MI', 'NA', 'PA', 'RM', 'TO', 'VE', 'RN']

for i in ruote:
    #df = a.carica_estrazioni(i)
    print(a.previsioni(i))